In [1]:
%pip install pyaudio numpy scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install vosk 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ------ --------------------------------- 2.4/14.0 MB 12.2 MB/s eta 0:00:01
   ---------------- ----------------------- 5.8/14.0 MB 14.1 MB/s eta 0:00:01
   ------------------------- -------------- 8.9/14.0 MB 14.2 MB/s eta 0:00:01
   ----------------------------------- ---- 12.3/14.0 MB 14.6 MB/s eta 0:00:01
   ---------------------------------------- 14.0/14.0 MB 14.6 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22557 sha256=63de34f7b19e88dc547a74a7e3512055c27a7ef19c1ba447e33db9e8cb45bcda
  Stored in directory: c:\users\om\appdata\local\pip\cache\wheels\7e\75\5b\e1d5c3756631e4bda806f6cc9640153b39484bb6f7b0b8def3
Successfully built srt
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install transformers torchaudio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pyaudio

In [5]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000

In [6]:
audio = pyaudio.PyAudio()

stream = audio.open(format=FORMAT, channels=CHANNELS,rate = RATE, input=True,frames_per_buffer=CHUNK)

In [10]:
from vosk import Model, KaldiRecognizer
import json

model = Model(r".\vosk-model-en-us-0.22")


In [11]:
recognizer =  KaldiRecognizer(model, RATE)

In [12]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


c:\Users\om\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import torch 

In [14]:
processor  = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import wave 
import numpy as np 

In [18]:
print("Start Speaking...")

while True: 
    data  = stream.read(CHUNK)

    if recognizer.AcceptWaveform(data):
        result = recognizer.Result()
        text = json.loads(result).get('text' , '')
        print(f"Text : {text}")


def transcribe_wav2vec(audio_chunk):
    input_values = processor(audio_chunk, return_tensors="pt", sampling_rate=RATE).input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription[0]

while True:
    data = stream.read(CHUNK)
    audio_chunk = np.frombuffer(data, dtype=np.int16).astype(np.float32) / 32768.0
    text = transcribe_wav2vec(audio_chunk)  
    print(f"Text: {text}")


Start Speaking...
Text : the
Text : the


KeyboardInterrupt: 